In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 500     # number of variables
num_ineq = 500    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msNonconvex
model = msNonconvex(num_var, num_ineq, timelimit=60)

## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 34831.17
Epoch 1, Validation Loss: 43.21
Epoch 2, Validation Loss: 35.74
Epoch 3, Validation Loss: 18.54
Epoch 4, Validation Loss: 26.63
Epoch 5, Validation Loss: 14.08
Epoch 6, Validation Loss: 8.38
Epoch 7, Validation Loss: 7.36
Epoch 8, Validation Loss: 10.26
Epoch 9, Validation Loss: 7.94
Epoch 10, Validation Loss: 16.69
Epoch 11, Validation Loss: 8.49
Epoch 12, Validation Loss: 7.56
Epoch 13, Validation Loss: 4.21
Epoch 14, Validation Loss: 2.82
Epoch 15, Validation Loss: 13.52
Epoch 16, Validation Loss: 4.26
Epoch 17, Validation Loss: 13.88
Epoch 18, Validation Loss: 5.60
Epoch 19, Validation Loss: 2.94
Epoch 20, Validation Loss: 1.20
Epoch 21, Validation Loss: 16.58
Epoch 22, Validation Loss: 1.21
Epoch 23, Validation Loss: 1.07
Epoch 24, Validation Loss: 0.96
Epoch 25, Validation Loss: 7.33
Epoch 26, Validation Loss: 3.42
Epoch 27, Validation Loss: 4.62
Epoch 28, Validation Loss: 0.90
Epoch 29, Validation Loss: 1.08
Epoch 30, Validation Loss: 6.26
Epoc

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lr_500-500.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [11:19<00:00,  1.47it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -0.709213          0.003185      0.010006
std       0.111958          0.025379      0.020855
min      -1.029298          0.000000      0.002000
25%      -0.749796          0.000000      0.003684
50%      -0.708204          0.000000      0.004529
75%      -0.653700          0.000000      0.007482
max       0.370585          0.314706      0.222358
Number of infeasible solution: 23


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 12499.07
Epoch 1, Validation Loss: 38.66
Epoch 2, Validation Loss: 23.40
Epoch 3, Validation Loss: 16.83
Epoch 4, Validation Loss: 34.34
Epoch 5, Validation Loss: 13.67
Epoch 6, Validation Loss: 12.04
Epoch 7, Validation Loss: 8.96
Epoch 8, Validation Loss: 16.09
Epoch 9, Validation Loss: 9.16
Epoch 10, Validation Loss: 4.83
Epoch 11, Validation Loss: 4.50
Epoch 12, Validation Loss: 10.73
Epoch 13, Validation Loss: 6.62
Epoch 14, Validation Loss: 3.60
Epoch 15, Validation Loss: 3.18
Epoch 16, Validation Loss: 6.19
Epoch 17, Validation Loss: 1.47
Epoch 18, Validation Loss: 1.09
Epoch 19, Validation Loss: 5.20
Epoch 20, Validation Loss: 1.87
Epoch 21, Validation Loss: 3.73
Epoch 22, Validation Loss: 1.36
Epoch 23, Validation Loss: 0.44
Epoch 24, Validation Loss: 1.26
Epoch 25, Validation Loss: 12.76
Epoch 26, Validation Loss: 1.45
Epoch 27, Validation Loss: 1.31
Epoch 28, Validation Loss: 0.30
Epoch 29, Validation Loss: 0.26
Epoch 30, Validation Loss: 0.81
Epoch

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lt_500-500.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [11:17<00:00,  1.48it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -1.637065          0.000588      0.010730
std       0.077686          0.010223      0.023958
min      -1.685198          0.000000      0.002048
25%      -1.656603          0.000000      0.003835
50%      -1.656603          0.000000      0.004636
75%      -1.655448          0.000000      0.007369
max      -0.782845          0.248091      0.273246
Number of infeasible solution: 5
